In [38]:
import os
import sys
import importlib
import json
import glob
import numpy as np
import tensorflow as tf

import util_recognition_network

# fn_ckpt = 'models/recognition_networks/arch2_taskA.ckpt-950000'
# fn_arch = 'models/recognition_networks/arch2.json'

fn_ckpt = 'models/recognition_networks/arch3_taskA.ckpt*'
fn_ckpt = glob.glob(fn_ckpt)[-1].replace('.index', '')
fn_arch = 'models/recognition_networks/arch3.json'

with open(fn_arch, 'r') as f_arch:
    list_layer_dict = json.load(f_arch)

fn_ckpt


'models/recognition_networks/arch3_taskA.ckpt-980000'

In [39]:
importlib.reload(util_recognition_network)

tf.reset_default_graph()

if 'taskA' in fn_ckpt:
    n_classes_dict = {"/stimuli/labels_binary_via_int": 517}
else:
    n_classes_dict = {"/stimuli/word_int": 794}

var_scope = 'brain_network'
with tf.variable_scope(var_scope):
    tensor_input = tf.placeholder(tf.float32, [None, 40, 20000, 1])
    tensor_output, tensors = util_recognition_network.build_network(
        tensor_input,
        list_layer_dict,
        n_classes_dict=n_classes_dict)

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=var_scope)
dict_vars_to_save = {}
for v in var_list:
    key = v.name.replace('brain_network/', '').replace('fc_top_0', 'fc_top')
    dict_vars_to_save[key] = v
    
saver_for_loading = tf.train.Saver(var_list=var_list, max_to_keep=0)
saver_for_saving = tf.train.Saver(var_list=dict_vars_to_save, max_to_keep=0)

with tf.Session() as sess:
    saver_for_loading.restore(sess, fn_ckpt)
    
    saver_for_saving.save(
        sess,
        save_path=os.path.join(os.path.dirname(fn_ckpt), 'NEW_' + os.path.basename(fn_ckpt)),
        global_step=None,
        latest_filename=None,
        meta_graph_suffix='meta',
        write_meta_graph=False,
        write_state=False,
        strip_default_attrs=False,
    )


INFO:tensorflow:Restoring parameters from models/recognition_networks/arch3_taskA.ckpt-980000


In [31]:
# dict_vars_to_save = {}

# for v in var_list:
#     key = v.name.replace('brain_network/', '').replace('fc_top_0', 'fc_top')
#     dict_vars_to_save[key] = v
#     print(key, v.name)


In [18]:
os.path.join(os.path.dirname(fn_ckpt), 'NEW_' + os.path.basename(fn_ckpt))

'models/recognition_networks/NEW_arch1_taskW.ckpt-360000'